In [ ]:
# default_exp train

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Training
> Functions that wrap the whole training processes

In [ ]:
# export
from grade_classif.models.plmodules import Normalizer, GradesClassifModel
from grade_classif.models.metrics import accuracy, precision, recall, f_1
from grade_classif.imports import *

In [ ]:
# export
def train_normalizer(hparams):
    model = Normalizer(hparams)
    model.fit()
    return model

Trains a `Normalizer` unet with parameters defined in `hparams`.

In [ ]:
# export
def train_classifier(hparams):
    model = GradesClassifModel(hparams, metrics=[accuracy] + [met for i in range(2) for met in (partial(precision, cat=i), partial(recall, cat=i), partial(f_1, cat=i))])
    model.fit()
    return model

Trains a `GradesClassifModel` for grade classifications with parameters defined in `hparams`.

In [ ]:
from grade_classif.params.parser import hparams

In [ ]:
train_classifier(hparams)

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
INFO:root:
            Name         Type Params
0     base_model   Sequential   11 M
1   base_model.0       Conv2d    9 K
2   base_model.1  BatchNorm2d  128  
3   base_model.2         ReLU    0  
4   base_model.3    MaxPool2d    0  
..           ...          ...    ...
70        head.3      Dropout    0  
71        head.4       Linear  262 K
72        head.5  BatchNorm1d    1 K
73        head.6      Dropout    0  
74        head.7       Linear    1 K

[75 rows x 3 columns]
                                                                         

Epoch 1:   0%|          | 0/3122 [00:00<?, ?batch/s]

Epoch 10:  76%|███████▌  | 2363/3122 [19:02<06:05,  2.08batch/s, batch_idx=2362, gpu=0, loss=0.317, v_num=0]

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
